In [1]:
from RecVAE.postgres_loader import PostgreSQLWrapper
import numpy as np

In [2]:
pg = PostgreSQLWrapper(PARAMS)
pg.connect()

Connected to PostgreSQL database successfully.


In [3]:
from svd_latent_builder import SvdLatentBuilder

latent_builder = SvdLatentBuilder(pg)
#latent_builder.load_df()
#latent_builder.process_df()
latent_builder.load_latent_space()
latent_builder.load_game_enum()
game_to_idx, idx_to_game = latent_builder.get_game_mapping()



/var/home/f0kes/dev/vae-cf-2/RecVAE/postgres_loader.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.conn)


In [4]:
game_to_idx, idx_to_game = latent_builder.get_game_mapping()

latent_builder.enumerate_users()
print(latent_builder.df['playtime_forever'].describe())
latent_builder.construct_latent_space()
latent_builder.save_latent_space()


/var/home/f0kes/dev/vae-cf-2/RecVAE/postgres_loader.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.conn)


TypeError: 'NoneType' object is not subscriptable

In [4]:
query = "select * from matched_snapshots"
df = pg.load_data_into_dataframe(query)
df.head()

,steam_id,game_id,now,past,diff
0,76561197960349209,440900,2024-04-21 04:48:18.513725,2024-04-20 04:47:57.257536,0
1,76561197960473494,730,2024-04-21 13:26:18.061331,2024-04-19 15:15:03.465502,-56
2,76561197960487064,431960,2024-04-21 01:24:38.504265,2024-04-20 00:45:12.662998,0
3,76561197960879154,730,2024-04-21 11:50:21.097109,2024-04-20 11:50:18.093278,0
4,76561197961138422,22370,2024-04-21 01:38:53.510231,2024-04-20 00:59:09.809807,0


In [5]:
df['time_diff'] = (df['now'] - df['past']).dt.days
df['norm_diff'] = df['diff'] / df['time_diff']

In [6]:
user_to_idx = {user: idx for idx, user in enumerate(df['steam_id'].unique())}
idx_to_user = {idx: user for user, idx in user_to_idx.items()}


In [7]:
from scipy.sparse import coo_matrix, csr_matrix

csr_df = csr_matrix((df['norm_diff'], (df['steam_id'].map(user_to_idx), df['game_id'].map(game_to_idx))),
                    shape=(len(user_to_idx), len(game_to_idx)))

MemoryError: Unable to allocate 138. GiB for an array with shape (362550, 51118) and data type float64

In [8]:


# Sum of elements greater than 0 in each row
row_sums = np.array([np.sum(row[row > 0]) for row in csr_df])




In [9]:
row_neg_sums = np.array([np.sum(row[row < 0]) for row in csr_df])


In [10]:
len(row_neg_sums)

361665

In [11]:
volumes = np.minimum(np.abs(row_sums), np.abs(row_neg_sums))


In [12]:
non_zero_indexes = [row.nonzero()[1] for row in csr_df]

In [36]:
from tqdm import tqdm
import heapq


def add_to_U(U, user, gain, loss, volume, heap, non_zero_index=None, perfect_game_latent=None, top_k=100):
    # Initialize user flow table U
    if gain == 0 or loss == 0 or volume == 0:
        return U, heap
    if heap is None:
        heap = []

    for i in non_zero_index:
        for j in non_zero_index:
            if user[i] < 0 < user[j]:
                mult = 1
                if perfect_game_latent is not None:
                    if i >= len(latent_builder.Vt) or j >= len(latent_builder.Vt):
                        continue
                    game_vector = (latent_builder.Vt[i] + latent_builder.Vt[j]) / 2  #latent_builder.Vt[j]  
                    mult = latent_builder.cosine_similarity(game_vector, perfect_game_latent)
                U[i, j] += volume * (user[j] / gain) * (user[i] / loss) * mult * mult * mult
                U[j, i] -= volume * (user[j] / gain) * (user[i] / loss) * mult * mult * mult
                num = U[i, j]
                if len(heap) < top_k:
                    heapq.heappush(heap, (num, (i, j), mult))
                elif num > heap[0][0] and (i, j) not in [x[1] for x in heap]:
                    heapq.heappushpop(heap, (num, (i, j), mult))

    return U, heap


#array of arrays of non-zero indexes

total = np.zeros((csr_df.shape[1], csr_df.shape[1]))
dota_idx = 570
poe_idx = 238960
binding_of_isaac_idx = 250900
#perfect_game_latent = latent_builder.game_avg_vector([dota_idx, poe_idx, binding_of_isaac_idx])
vt = latent_builder.Vt

perfect_game_latent = np.mean([vt[game_to_idx[i]] for i in [dota_idx, poe_idx, binding_of_isaac_idx]], axis=0)
heap = []
for i in tqdm(range(csr_df.shape[0])):
    #need to convert csr_df[i] to dense array
    csr_df_i = csr_df[i].toarray().flatten()
    total, heap = add_to_U(total,
                           csr_df_i,
                           row_sums[i],
                           row_neg_sums[i],
                           volumes[i],
                           heap,
                           non_zero_index=non_zero_indexes[i],
                           perfect_game_latent=perfect_game_latent)

heap




100%|██████████| 361665/361665 [00:40<00:00, 8917.72it/s] 


[(1014.6569128359662, (5327, 35330), 0.9959252424230937),
 (1020.65842088572, (10280, 10276), 0.9889767293469289),
 (1015.5269337912019, (31634, 27581), 0.9960560453823212),
 (1023.4624890160163, (7963, 16852), 0.9952225821505052),
 (1029.806106849216, (15696, 31081), 0.9495894623045519),
 (1021.9972147423794, (24815, 27581), 0.9929249061643799),
 (1028.6704650205836, (33314, 26200), 0.9952154866404622),
 (1038.4016081731218, (7779, 27663), 0.9916147929866296),
 (1058.3249698171944, (16852, 47642), 0.9959630759606137),
 (1074.9968763665347, (47715, 15696), 0.9929442902961206),
 (1036.886893016516, (7963, 26330), 0.9953086087754817),
 (1058.2213412692665, (7779, 36425), 0.9911413829395279),
 (1025.9259181816922, (30764, 28256), 0.5612731854100379),
 (1029.9577672885625, (16852, 26311), 0.9956809065355577),
 (1044.8365573689698, (32629, 15602), 0.9957481916059134),
 (1071.689408436187, (7779, 32922), 0.994377428565048),
 (1044.2312496682316, (30748, 10276), 0.9907987549984341),
 (1141.54

In [38]:
#convert heap[1] to game ids
flows = [(x[0], (idx_to_game[x[1][0]], idx_to_game[x[1][1]]), x[2]) for x in heap]
#sort flows by x[0]
flows = sorted(flows, key=lambda x: x[0], reverse=True)
flows

[(8008.8763432990245, (238960, 570), 0.995619931306679),
 (5544.686118387973, (553850, 570), 0.989941675172233),
 (5211.842237301942, (221100, 570), 0.9898345779366735),
 (4806.354545017273, (440, 570), 0.9924108403849448),
 (4238.1336661338955, (2524890, 570), 0.989894858803113),
 (4202.012910373937, (2881650, 570), 0.9886099623296386),
 (4094.2759641337334, (739630, 570), 0.990610098414417),
 (3850.3932794547536, (553850, 1085660), 0.9956123338992495),
 (3349.3336118295833, (1172470, 570), 0.9901137387862),
 (3297.5260650252308, (230410, 570), 0.9906754953911481),
 (2866.2648775123243, (480, 570), 0.8780804359231708),
 (2836.239001703016, (2881650, 578080), 0.9753274736969987),
 (2733.016263569747, (1566690, 553850), 0.9922732482495595),
 (2481.5920451637335, (1172470, 578080), 0.9762691220575871),
 (2217.105117474475, (1818450, 570), 0.9937609782704937),
 (2134.776105416886, (1966720, 570), 0.9955594867888),
 (2073.3755163393944, (2881650, 1172470), 0.9925784354532269),
 (1957.52881

NameError: name 'total' is not defined